<a href="https://colab.research.google.com/github/radhika1112/Breaking-News-Detection1/blob/master/Radhika_EchoBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Radhika Agrawal,
Msc Data Science, 
University of Glasgow**

# Echobox Data Science Challenge 2019
# Breaking News Detection







### Downloading libraries

In [0]:
import spacy
import pandas as pd

# Load the medium english model. 
# We will use this model to get embedding features for tokens later.
!python -m spacy download en_core_web_md

import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 95.4MB 41.8MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=cdf95d1a77b75c097420eeae67b79c25906567deae3d90eed5191577d2f571b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-ij9s4sqx/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading packa

True

In [0]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
!pip install tqdm

In [0]:
from sklearn.cluster import DBSCAN

In [0]:
import spacy

In [0]:
from tqdm import tqdm

### Loading the dataset

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd 
root_path = 'gdrive/My Drive/EchoboxCSV.csv/'
df=pd.read_csv('gdrive/My Drive/EchoboxCSV.csv')
df.shape

(13022, 6)

### Analysing the data

In [0]:
df.head(10)

,ArticleId,ArticleURL,ArticleTitle,ArticleDescription,ArticlePublishedTime,EventId
0,5cd7ed707ddacd3b2b3b549e,https://www.bbc.co.uk/news/uk-england-suffolk-...,Lowestoft sea wall fall cyclist rescued by friend,Coastguards praise the boy's friend for his ac...,1557653530,0.0
1,5cd7e83beb96a44751294217,https://www.highsnobiety.com/p/met-gala-best-c...,The Met Gala & ‘Game of Thrones’ Feature in Th...,"Once again, our ever-sarcastic readership have...",1557653563,0.0
2,5cd7e99a8e662d1e4435cb3d,https://www.mirror.co.uk/news/uk-news/boy-dies...,Boy dies on prom day after allergic reaction t...,Joe Dale's family have spoken out about losing...,1557653574,0.0
3,5cd7f6dd7ddacd3b2b3b56ab,https://www.independent.co.uk/voices/paddy-jac...,Paddy Jackson’s return to Rugby is yet more pr...,He may have been found not guilty of rape last...,1557653588,0.0
4,5cd7e89c8e662d1e4435cb13,https://www.standard.co.uk/showbiz/celebrity-n...,BAFTA TV Awards 2019: Stars prepare for glitzy...,Stars are preparing for Sunday night s TV Baft...,1557653610,0.0
5,5cd7e8ad8e662d1e4435cb18,https://newsinfo.inquirer.net/1117187/look-poe...,LOOK: Poe shares son’s fave photo of her on Mo...,"Sen. Grace Poe, who is running for re-election...",1557653627,0.0
6,5cd7eb788e662d1e4435cb9c,https://www.mirror.co.uk/sport/football/news/p...,Pep Guardiola 'bans Man City staff from checki...,Man City will win the Premier League with vict...,1557653639,0.0
7,5cd8124b7ddacd3b2b3b5cda,https://virginradio.co.uk/highlights/lewis-cap...,"Lewis Capaldi, Olivia Wilde, Mena Massoud & MO...",HighlightsThe cavalcade of stars continued thi...,1557653674,0.0
8,5cd7e90e7ddacd3b2b3b53b4,https://www.smh.com.au/sport/soccer/wanderers-...,Wanderers face leadership crisis as captain Ha...,Western Sydney Wanderers captain Brendan Hamil...,1557653689,0.0
9,5cd7ea8c7ddacd3b2b3b540b,https://ahvalnews.com/berat-albayrak/turkeys-f...,Turkey's finance minister signals inflation ra...,Turkey's finance minister on Sunday said the c...,1557653697,0.0


### Removing Nans

In [0]:
df.EventId.fillna(0,inplace=True)

In [0]:
df.shape

(13022, 6)

### Removing Duplicates

In [0]:
df=df.drop_duplicates(subset='ArticleTitle').reset_index(drop=True)
df.shape

(13022, 6)

In [0]:
stop_words = set(nltk.corpus.stopwords.words('english'))

In [0]:
nlp = spacy.load('en')


### Tokenising and normalising

### Removing Stopwords, extraneous spaces, lower case, and punctuations.

In [0]:
def normalized_tokenizer(string):
  tokens = []
  normalized_tokens = []
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha)):
      if token.text not in stop_words: #stopword removal
        normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

In [0]:
normalized_tokenizer("I am working on Echobox challenGE.., I want this InternShip!!:)")

['i', 'working', 'echobox', 'challenge', 'i', 'want', 'internship']

### Converting the article title into vector form.

Used SpaCy’s pre-trained word embeddings model en_core to give meaning to the words/sentences. Word2Vec, FastText or even GloVe could also be used.
By default, SpaCy considers a sentence’s vector as the average between every word’s vector. 


In [0]:
sentence_vector={}
docs=[]
for title in tqdm(df.ArticleTitle):
  doc = nlp(title)
  docs.append(doc)
  sentence_vector.update({title:doc.vector})
  
sentences=list(sentence_vector.keys())
vectors=list(sentence_vector.values())

100%|██████████| 13022/13022 [02:24<00:00, 89.87it/s]


In [0]:
sentences

In [0]:
import numpy as np

### DBSCAN
Now, that our data is preprocessed and is in the correct input vector format, We have to cluster them based on the similarities of the Article Title.
Cosine Similarity is used to form clusters.

In [0]:
x=np.array(vectors)

num_classes={}

for i in tqdm(np.arange(0.001,1,0.002)):
  dbscan=DBSCAN(eps=i,min_samples=2,metric='cosine').fit(x)
  num_classes.update({i:len(pd.Series(dbscan.labels_).value_counts())})
  
dbscan=DBSCAN(eps=0.08,min_samples=2,metric='cosine').fit(x)

100%|██████████| 500/500 [21:29<00:00,  3.44s/it]



**We get output of as the int labels and the similar sentences with same labels.**

**The labels that are -1 have no coreelation with the other sentences, the labels with 0 do not belong to a cluster.**

In [0]:
results=pd.DataFrame({'label':dbscan.labels_,'sent':sentences})
event_df=df[df.ArticleTitle.isin(results.sent)][['ArticleURL','ArticlePublishedTime','ArticleTitle','ArticleDescription']]
event_df['ArticlePublishedTime']=pd.to_datetime(event_df.ArticlePublishedTime)
# for r, i in zip(results.label, for i in range (1,506)):
#   event_df['id']= i
  
  
# event_df=event_df.sort_values(by='ArticlePublishedTime')

### Example Result

**We can see that cluster label '338' has both the article similar as they talk about the same topic ie. "Doris Day dies aged 97"**

In [0]:
example_result=results[results.label==338].sent.tolist()
ex_event_df=df[df.ArticleTitle.isin(example_result)][['ArticlePublishedTime','ArticleTitle','ArticleDescription']]
ex_event_df['ArticlePublishedTime']=pd.to_datetime(event_df.ArticlePublishedTime)
# event_df=event_df.sort_values(by='ArticlePublishedTime')

In [0]:
example_result

['Doris Day dies aged 97',
 'Actress Doris Day dies aged 97',
 'Hollywood star Doris Day dies at 97',
 'American actress Doris Day dies aged 97',
 'Screen legend Doris Day dies at 97',
 'Screen legend Doris Day dies aged 97',
 'Screen legend Doris Day dies, aged 97',
 'Hollywood star Doris Day dies aged 97',
 'Hollywood legend Doris Day dies aged 97',
 'Hollywood icon Doris Day dies at 97']

In [0]:
ex_event_df

,ArticlePublishedTime,ArticleTitle,ArticleDescription
6880,1970-01-01 00:00:01.557752179,Doris Day dies aged 97,NaN
6959,1970-01-01 00:00:01.557752648,Actress Doris Day dies aged 97,"Hollywood star Doris Day, whose films made her..."
7040,1970-01-01 00:00:01.557753336,Hollywood star Doris Day dies at 97,NaN
7136,1970-01-01 00:00:01.557754139,American actress Doris Day dies aged 97,NaN
7137,1970-01-01 00:00:01.557754141,Screen legend Doris Day dies at 97,"With her lilting contralto, wholesome blonde b..."
7186,1970-01-01 00:00:01.557754609,Screen legend Doris Day dies aged 97,"Doris Day, whose wholesome screen presence sto..."
7187,1970-01-01 00:00:01.557754609,"Screen legend Doris Day dies, aged 97","Doris Day, one of the most beloved and recogni..."
7392,1970-01-01 00:00:01.557756308,Hollywood star Doris Day dies aged 97,NaN
7686,1970-01-01 00:00:01.557758720,Hollywood legend Doris Day dies aged 97,Singer-turned-actress Doris Day was one of the...
7964,1970-01-01 00:00:01.557760828,Hollywood icon Doris Day dies at 97,NaN


In [0]:
results.head(10)

,label,sent
0,-1,Lowestoft sea wall fall cyclist rescued by friend
1,0,The Met Gala & ‘Game of Thrones’ Feature in Th...
2,1,Boy dies on prom day after allergic reaction t...
3,-1,Paddy Jackson’s return to Rugby is yet more pr...
4,-1,BAFTA TV Awards 2019: Stars prepare for glitzy...
5,0,LOOK: Poe shares son’s fave photo of her on Mo...
6,0,Pep Guardiola 'bans Man City staff from checki...
7,0,"Lewis Capaldi, Olivia Wilde, Mena Massoud & MO..."
8,-1,Wanderers face leadership crisis as captain Ha...
9,-1,Turkey's finance minister signals inflation ra...


### Lets check the top 50 cluster and their sizes

Total number of clusters formed is 506

In [0]:
results['label'].value_counts().nlargest(50)

-1      10022
 0       1769
 5         28
 150       13
 114       11
 47        11
 338       10
 23         9
 70         6
 42         6
 289        6
 191        6
 339        6
 8          6
 24         6
 166        6
 27         5
 72         5
 174        5
 167        5
 277        5
 197        5
 304        5
 29         4
 222        4
 93         4
 161        4
 173        4
 1          4
 15         4
 129        4
 461        4
 63         4
 37         4
 25         4
 268        4
 362        4
 19         4
 40         4
 314        4
 180        4
 140        4
 59         3
 233        3
 241        3
 273        3
 51         3
 485        3
 28         3
 345        3
Name: label, dtype: int64

### The Final DataFrame

In [0]:
event_df.head(10)

,ArticleURL,ArticlePublishedTime,ArticleTitle,ArticleDescription,EventId
0,https://www.bbc.co.uk/news/uk-england-suffolk-...,1970-01-01 00:00:01.557653530,Lowestoft sea wall fall cyclist rescued by friend,Coastguards praise the boy's friend for his ac...,0.0
1,https://www.highsnobiety.com/p/met-gala-best-c...,1970-01-01 00:00:01.557653563,The Met Gala & ‘Game of Thrones’ Feature in Th...,"Once again, our ever-sarcastic readership have...",0.0
2,https://www.mirror.co.uk/news/uk-news/boy-dies...,1970-01-01 00:00:01.557653574,Boy dies on prom day after allergic reaction t...,Joe Dale's family have spoken out about losing...,0.0
3,https://www.independent.co.uk/voices/paddy-jac...,1970-01-01 00:00:01.557653588,Paddy Jackson’s return to Rugby is yet more pr...,He may have been found not guilty of rape last...,0.0
4,https://www.standard.co.uk/showbiz/celebrity-n...,1970-01-01 00:00:01.557653610,BAFTA TV Awards 2019: Stars prepare for glitzy...,Stars are preparing for Sunday night s TV Baft...,0.0
5,https://newsinfo.inquirer.net/1117187/look-poe...,1970-01-01 00:00:01.557653627,LOOK: Poe shares son’s fave photo of her on Mo...,"Sen. Grace Poe, who is running for re-election...",0.0
6,https://www.mirror.co.uk/sport/football/news/p...,1970-01-01 00:00:01.557653639,Pep Guardiola 'bans Man City staff from checki...,Man City will win the Premier League with vict...,0.0
7,https://virginradio.co.uk/highlights/lewis-cap...,1970-01-01 00:00:01.557653674,"Lewis Capaldi, Olivia Wilde, Mena Massoud & MO...",HighlightsThe cavalcade of stars continued thi...,0.0
8,https://www.smh.com.au/sport/soccer/wanderers-...,1970-01-01 00:00:01.557653689,Wanderers face leadership crisis as captain Ha...,Western Sydney Wanderers captain Brendan Hamil...,0.0
9,https://ahvalnews.com/berat-albayrak/turkeys-f...,1970-01-01 00:00:01.557653697,Turkey's finance minister signals inflation ra...,Turkey's finance minister on Sunday said the c...,0.0


### Getting the mean Vector

In [0]:
def get_mean_vector(sents):
  a=np.zeros(96)
  for sent in sents:
    a=a+nlp(sent).vector
  return a/len(sents)
  

In [0]:
def get_central_vector(sents):
  vecs=[]
  for sent in sents:
    doc=nlp(ArticleTitle)
    vecs.append(doc.vector)
  mean_vec=get_mean_vector(sents)
  index=pairwise_distance_argmin_min(np.array([mean_vec]),vecs)[0][0]
  return sents[index]
  

In [0]:
l=get_mean_vector(results['sent'])
print(l)

[ 1.65051925 -1.5371054  -0.99965313 -1.71156383 -0.65126257  0.55436463
  2.69546819  0.91861703 -0.68392492  1.24070817 -0.12683924 -0.1585369
 -1.48919119 -1.22342266 -0.24514404 -0.53235199  0.44680361  1.30903564
  0.81043549  1.54462343 -1.33128678 -0.92360737  1.07334576  1.88306888
 -0.29855695  0.84668891  0.66915921 -0.80656889  1.07871278  2.62134186
  1.21903926 -0.65371054 -0.57431731 -0.33382327  1.65673981  0.78495114
 -0.75869497  1.18659917 -0.40910358  0.2058669   0.19911874  1.57140003
  0.62300667  0.14321044  0.10085483 -0.66821382 -1.51804585  0.39318142
 -0.19692304 -0.23873555 -1.58125683 -0.08279669 -1.42581674 -0.60242489
  0.32430983 -0.33649655  0.05084067  0.70206254 -1.29522077 -0.39100278
 -0.80925884 -0.3543978   1.62086812 -0.45322847 -0.91090165  1.66027422
  0.05358627 -0.18238331 -0.76154386  0.00847259  0.79360505 -0.20622177
  0.45041912  1.45102963  0.15825019  0.58174042 -0.2370958  -0.93666328
 -0.37323417 -1.2908154  -0.98866929 -1.24871372 -1.